In [4]:
import importlib
import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (20, 10)

In [5]:
archivos_a_leer = [
    "modelo_logistic.pickle",
    "modelo_naive_bayes.pickle",
    "modelo_arbol.pickle",
    "modelo_random_forest.pickle",
    "modelo_gradient_boosting.pickle",
]

modelos = {}
for nombre_archivo in archivos_a_leer:
    with open(nombre_archivo, 'rb') as file:
        modelo = pickle.load(file)
        modelos[nombre_archivo.split('.')[0]] = modelo 